In [11]:
import numpy as np
import scipy as sp
import pandas as pd
import pytraj as pt
import nglview as nv

import matplotlib
from matplotlib import pyplot as plt

import cphmdanalysis as cphmda
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [28]:
nRuns=2
runs=np.arange(2)
sysName='R206H'
with open('pH_List.txt','r') as pHlistFile:
    rep_pH_vals=list(map(float,pHlistFile.readline().split(', ')))
phList=rep_pH_vals*nRuns
nReps=len(rep_pH_vals)
lambdaFileNames=['rep%02g.%02g.lambda'%(
                    #sysName,
                    irep+1,irun) \
                 for irun in runs \
                 for irep in np.arange(len(rep_pH_vals))]
lambdaFiles=[cphmda.lambda_file(filename) for filename in lambdaFileNames]

for lambda_file in lambdaFiles:
    lambda_file.compute_s_values()

pka_data=cphmda.compute_pkas(phList,lambdaFiles) #wrapped_lambda_files)

tempStruc=pt.load('%s.min.rst7'%sysName,top='%s.parm7'%sysName)
orig_struc=pt.load("../../../structures/WT.pdb")

standard_pKa_dict={
    "AS2":3.71,"GL2":4.15,"HIP":6.04,"CYS":8.14,"LYS":10.67}

pKaTable=pd.DataFrame([(tempStruc.top.residue(res-1).name,
 (tempStruc.top.residue(res-1)).original_resid,
 (orig_struc.top.residue(res-1)).original_resid,
  "%.3f"%(
      pka_data[ires][2]),
  "%.3f"%(
      pka_data[ires][0])) for ires,res in enumerate(lambdaFiles[0].titrreses)],
            columns=['ResName','Resid','PDB_Resid','Predicted_pKa','Hill_Coef'])
pKaTable["Standard_pKa"]=pKaTable.ResName.apply(
    lambda x: standard_pKa_dict[x])
pKaTable["Delta_pKa"]=pd.Series(pKaTable.Predicted_pKa,dtype=float)- \
    pd.Series(pKaTable.Standard_pKa,dtype=float)
pKaTable.to_csv('pKa_DataTable.csv',index=False)
goodRows=np.sum((pKaTable.dropna().Standard_pKa!='nan'))
totalRows=pKaTable.shape[0]
print('Total Number of Residues: ',
      totalRows,
      '\nNumber of succesfully titrated residues: ',
      goodRows,
      '\nNumber of unsuccesfully titrated residues: ',
      totalRows-goodRows)
pKaTable.head()

Total Number of Residues:  96 
Number of succesfully titrated residues:  70 
Number of unsuccesfully titrated residues:  26


/Users/wbotellosmith/anaconda2/envs/py3/lib/python3.7/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


ResName  Resid  PDB_Resid Predicted_pKa Hill_Coef  Standard_pKa  Delta_pKa
0     AS2      6        177         2.659     0.458          3.71     -1.051
1     AS2     11        182         2.696     0.635          3.71     -1.014
2     AS2     14        185         3.446     2.126          3.71     -0.264
3     HIP     15        186         6.628     0.364          6.04      0.588
4     CYS     17        188        11.429     0.305          8.14      3.289

In [29]:
@interact_manual
def show_residue_titration_data(
    resList="HIP CYS LYS",
    dropIncompleteRows=True,
    showStandard_pKa=True,
    showHillCoef=True,
    useHillScore=False,
    hillScoreCut=(0,1,.01),
    sortColumns='Delta',
    ascendSort=True,
    absDeltaCut=(0,5,.05)):
    outFrame=pKaTable[pKaTable.ResName.isin(resList.split())][np.concatenate([
        ['ResName','PDB_Resid'],
        [colname for colname in pKaTable.columns \
         if ('Predicted_pKa' in colname) | ('Hill' in colname) | ('Standard' in colname)]])]
    outFrame['Delta']=outFrame['Predicted_pKa'].map(float)-outFrame['Standard_pKa'].map(float)
    outFrame['Hill_Coef']=outFrame.Hill_Coef.map(lambda x: float(x))
    if dropIncompleteRows:
        outFrame=outFrame.dropna()
    if absDeltaCut > 0.0:
        outFrame=outFrame[outFrame.Delta.abs()>absDeltaCut]
    if useHillScore:
        if hillScoreCut > 0.0:
            outFrame['Hill_Check']=0
        for colName in outFrame.columns:
            if 'Hill' in colName:
                outFrame[colName]=1.0/np.exp(np.abs(np.log(outFrame[colName])))
                if hillScoreCut > 0.0:
                    outFrame.Hill_Check=outFrame.Hill_Check + outFrame[colName]>hillScoreCut
        if hillScoreCut > 0.0:
            outFrame=outFrame[outFrame.Hill_Check > 0]
            outFrame=outFrame[outFrame.columns[:-1]]
    outFrame=outFrame.sort_values(sortColumns.split(),ascending=ascendSort)
    if not showStandard_pKa:
        outFrame=outFrame[[
            colName for colName in outFrame.columns \
            if not ('Standard' in colName)]]
    if not showHillCoef:
        outFrame=outFrame[[
            colName for colName in outFrame.columns \
            if not ('Hill' in colName)]]
    return outFrame

interactive(children=(Text(value='HIP CYS LYS', description='resList'), Checkbox(value=True, description='drop…